In [1]:
import numpy as np
import os
from scipy.stats import norm
import math
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve, root
import datetime
import copy
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


In [11]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [82]:
# 2019
lambdas_19 = pd.Series(data = np.array([0.23142747, 0.22392409, 0.20841536, 0.20501973, 0.20180459, 0.19610248, 0.19278537, 0.19046998, 0.18773363, 0.1848987, 0.1818808, 0.18194688, 0.18148715, 0.18106108, 0.18025282, 0.1809604, 0.18489099, 0.18551875, 0.18541091]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2020
lambdas_20 = pd.Series(data = np.array([0.15802172,0.19436255,0.19895045,0.20514595,0.20869606,0.22235965,0.22893855,0.22993542,0.22757478,0.22469209,0.22192999,0.21928466,0.21810217,0.2159931,0.2164775,0.22009477,0.22697407,0.22733004,0.22072845]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2021
lambdas_21 = pd.Series(data = np.array([0.16328444,0.17134324,0.17588378,0.18259829,0.18762413,0.19757024,0.19972312,0.19790383,0.19491882,0.19187272,0.18962228,0.18776167,0.18514741,0.18400098,0.1836981,0.18566882,0.19089332,0.19410368,0.19036148]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2022
lambdas_22 = pd.Series(data = np.array([0.44633652,0.46375938,0.46842593,0.46920615,0.46981876,0.4386397,0.40159346,0.37250602,0.34866545,0.33646934,0.32538128,0.32228333,0.31854968,0.31827933,0.33548185,0.36918706,0.41590999,0.41530556,0.39033859]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2023
lambdas_23 = pd.Series(data = np.array([0.2613744,0.28752945,0.30563777,0.31611101,0.32526963,0.34324312,0.33804256,0.32556478,0.30885869,0.29948776,0.28968739,0.28191525,0.27534523,0.27105808,0.27166874,0.27740328,0.28880094,0.29935596,0.29592533]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

In [83]:
# latex_width = 6.69423 inches
w = 6.69423 * 7.6/10
print(w)
golden_ratio = (5**.5 - 1) / 2
h = 4.685961000000001* golden_ratio
h

5.0876148


2.8960831679564465

In [84]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0.15,0.5)
ax.set_yticks(np.arange(0.15, 0.55,0.05))
ax.set_xlim(0, 30)
ax.set_xticks(np.arange(0,32,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\lambda(t)$')

plt.step(lambdas_23.index, lambdas_23, label = '2023', color = 'b')
plt.step(lambdas_22.index, lambdas_22, label = '2022', color = 'r')
plt.step(lambdas_21.index, lambdas_21, label = '2021', color = 'g')
plt.step(lambdas_20.index, lambdas_20, label = '2020', color = '#ff7f0e')
plt.step(lambdas_19.index, lambdas_19, label = '2019', color = 'm')

leg = plt.legend(bbox_to_anchor = (1.23, 1.029), loc = 'upper right', fontsize=9)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_boxstyle('Round', pad=0)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/lambdas_2.pgf')

## Calibration 3

In [85]:
# 2019
lambd_calibrated_19 = pd.Series(data = np.array([0.23432757, 0.22832776, 0.21239419, 0.20989871, 0.20458665, 0.19417452, 0.19162511, 0.18950253, 0.18643243, 0.18467962, 0.18448355, 0.18315475, 0.1830693, 0.18275957, 0.18249228,0.18304122, 0.18442, 0.18276165, 0.18336802]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
b_calibrated_19 = pd.Series(data = np.array([0.01899839, 0.01324599, 0.01135441, 0.00799124, 0.00687906, 0.00567754, 0.00523664, 0.00548982, 0.06432817, 0.09185644, 0.11429008, 0.12910222, 0.14095165, 0.1504578, 0.1500195, 0.14458935, 0.12713708, 0.21141059, 0.29543713]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
eta_calibrated_19 = pd.Series(data = np.array([2.80953275, 2.28553246, 1.92811350, 1.93820309, 1.68684801, 1.21198740, 1.28167217, 1.29083759, 1.25464691, 1.36693343, 1.68367180, 1.53955988, 1.59536753, 1.62207769, 1.72462833, 1.74669833, 1.29855035, 1.42101947e-02, 8.69731241e-04]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2020
lambd_calibrated_20 = pd.Series(data = np.array([0.16320032, 0.20354256, 0.20783627, 0.21215619, 0.21239257, 0.22225025, 0.22964738, 0.22921283, 0.22290858, 0.22178357, 0.22031429, 0.21856191, 0.21767211, 0.21537081, 0.21753519, 0.22215723, 0.22653692, 0.23040852, 0.22648443]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
b_calibrated_20 = pd.Series(data = np.array([0.25663186, 0.28690968, 0.19198857, 0.01281051, 0.16157636, 0.00663341, 0.08698882, 0.10306572, 0.11246155, 0.12172789, 0.12692493, 0.13315538, 0.13678724, 0.13977688, 0.13943112, 0.13564953, 0.12801558, 0.12860921, 0.13419064]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
eta_calibrated_20 = pd.Series(data = np.array([3.40128264, 3.06260558, 2.51195877, 2.13172695, 1.79250953, 1.50399127, 1.54373779, 1.47445375, 1.27126844, 1.35685672, 1.42138193, 1.46793091, 1.48325137, 1.4699439 , 1.5764693, 1.65124641, 1.47441417, 1.77503995, 2.05441658]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2021
lambd_calibrated_21 = pd.Series(data = np.array([0.16246652, 0.16951984, 0.1725005 , 0.17812848, 0.18949832, 0.19736336, 0.20056205, 0.1974455 , 0.1909636, 0.18947078, 0.18836681, 0.18719459, 0.18481664, 0.18382116, 0.18447391, 0.187066, 0.18993899, 0.19612096, 0.19484798]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
b_calibrated_21 = pd.Series(data = np.array([1.28999918, 1.2998979 , 1.31486662, 1.32999832, 1.13315251, 0.70772095, 0.57534925, 0.47167497, 0.38281141, 0.36311661, 0.34131284, 0.31877607, 0.29744674, 0.26819894, 0.26364688, 0.2446193 , 0.21581184, 0.20789896, 0.20751869]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
eta_calibrated_21 = pd.Series(data = np.array([0.81810631, 0.72850604, 0.74740941, 0.83704278, 1.63601765, 1.46253091, 1.52486403, 1.45325872, 1.23647065, 1.32896457, 1.40056102, 1.44624521, 1.46307495, 1.47631064, 1.55352, 1.61681504, 1.40152722, 1.73070909, 2.04864653]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2022
lambd_calibrated_22 = pd.Series(data = np.array([0.44881751, 0.46793747, 0.47171461, 0.46822535, 0.46415754, 0.43598409, 0.40382044, 0.37361462, 0.34442717, 0.33509187, 0.33039076, 0.32330333, 0.31949234, 0.31850325, 0.33717737, 0.37078997, 0.41136205, 0.4029509, 0.39076462]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
b_calibrated_22 = pd.Series(data = np.array([0.79006455, 0.74063312, 0.62061772, 0.56137939, 0.49810169, 0.42822463, 0.392732, 0.34145724, 0.28016858, 0.26052589, 0.26121412, 0.21071532, 0.18301017, 0.15299069, 0.14985264, 0.14255694, 0.13602369, 0.13538185, 0.1453642]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
eta_calibrated_22 = pd.Series(data = np.array([2.51536348, 2.27926297, 2.12571487, 1.94411668, 1.79125609, 1.88594861, 2.04604274, 2.0206388 , 1.78487811, 1.91967453, 2.26938125, 2.06745737, 2.07563793, 2.03938733, 2.15317587, 2.1675717 , 1.70069733, 0.25004989, 2.17300551]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

# 2023
lambd_calibrated_23 = pd.Series(data = np.array([0.26566008, 0.29352139, 0.31195871, 0.32140308, 0.32648644,0.34152468, 0.33766059, 0.32362113, 0.30234794, 0.29614089, 0.28883064, 0.28287323, 0.27747187, 0.27377777, 0.27638367, 0.28256603, 0.28975099, 0.30131673, 0.29687095]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
b_calibrated_23 = pd.Series(data = np.array([0.0183784 , 0.17372779, 0.1887257 , 0.19562428, 0.19653525, 0.2031521 , 0.19730479, 0.19342452, 0.19038863, 0.18365973, 0.17671099, 0.16783673, 0.15778136, 0.14615985, 0.1413279, 0.13187765, 0.11659032, 0.11282423, 0.11456432]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])
eta_calibrated_23 = pd.Series(data = np.array([3.09401321, 2.30990128, 1.97551876, 1.80865336, 1.59230674, 1.48578926, 1.52684785, 1.47954186, 1.32609028, 1.42419107, 1.5071296 , 1.57420393, 1.62346108, 1.65338718, 1.74639681, 1.78302146, 1.59995972, 1.66621696, 1.6197562]), 
                    index = [1/12,3/12,6/12,9/12,1,2,3,4,5,6,7,8,9,10,12,15,20,25,30])

In [86]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0.15,0.5)
ax.set_yticks(np.arange(0.15, 0.55,0.05))
ax.set_xlim(0, 30)
ax.set_xticks(np.arange(0,32,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\lambda(t)$')

plt.step(lambd_calibrated_23.index, lambd_calibrated_23, label = '2023', color = 'b')
plt.step(lambd_calibrated_22.index, lambd_calibrated_22, label = '2022', color = 'r')
plt.step(lambd_calibrated_21.index, lambd_calibrated_21, label = '2021', color = 'g')
plt.step(lambd_calibrated_20.index, lambd_calibrated_20, label = '2020', color = '#ff7f0e')
plt.step(lambd_calibrated_19.index, lambd_calibrated_19, label = '2019', color = 'm')

leg = plt.legend(bbox_to_anchor = (1.23, 1.029), loc = 'upper right', fontsize=9)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_boxstyle('Round', pad=0)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/lambdas_3.pgf')

In [90]:
# latex_width = 6.69423 inches
# latex_height = 
w = 6.69423 * 5.21/10
golden_ratio = (5**.5 - 1) / 2
h = 6.69423 * 5/6 * golden_ratio
h

3.4477180570910075

In [91]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0,1.4)
ax.set_yticks(np.arange(0,1.6,0.2))
ax.set_xlim(0, 30)
ax.set_xticks(np.arange(0,32,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$b(t)$')

plt.step(b_calibrated_23.index, b_calibrated_23, label = '2023', color = 'b')
plt.step(b_calibrated_22.index, b_calibrated_22, label = '2022', color = 'r')
plt.step(b_calibrated_21.index, b_calibrated_21, label = '2021', color = 'g')
plt.step(b_calibrated_20.index, b_calibrated_20, label = '2020', color = '#ff7f0e')
plt.step(b_calibrated_19.index, b_calibrated_19, label = '2019', color = 'm')

plt.legend(frameon=False, loc = 'upper right', fontsize=9)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/bs_3.pgf')

In [92]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0,3.5)
ax.set_yticks(np.arange(0,4,0.5))
ax.set_xlim(0, 30)
ax.set_xticks(np.arange(0,32,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\eta(t)$')

plt.step(eta_calibrated_23.index, eta_calibrated_23, label = '2023', color = 'b')
plt.step(eta_calibrated_22.index, eta_calibrated_22, label = '2022', color = 'r')
plt.step(eta_calibrated_21.index, eta_calibrated_21, label = '2021', color = 'g')
plt.step(eta_calibrated_20.index, eta_calibrated_20, label = '2020', color = '#ff7f0e')
plt.step(eta_calibrated_19.index, eta_calibrated_19, label = '2019', color = 'm')

plt.legend(frameon=False,  loc = 'upper right', fontsize=9)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/etas_3.pgf')